In [7]:
from lib import rms_db
import pandas as pd

### 1. help_region_lu

In [8]:
rms_db.select("""
    with a as (
            select ROW_NUMBER() over(order by DC_KEY, INV_KEY, BI_KEY) as region_id,
                DC_KEY, INV_KEY, a.BI_KEY
            from 
                (select distinct DC_KEY, INV_KEY, BI_KEY from huvgeous) a 
        ),
        b as (
            select ROW_NUMBER() over(order by BI_KEY) as bi, BI_KEY
            from 
                (select distinct BI_KEY from huvgeous) a        
        )
    select a.region_id + b.bi * 256, a.DC_KEY, a.INV_KEY, a.BI_KEY, b.bi
    from a join b on a.BI_KEY = b.BI_KEY""").to_csv('working/help_region_lu.csv', index=False, header=True)

### 2.help_occ_lu

In [21]:
rms_db.select("""
    select a.occ_id + b.bi* 256 as occ_id, a.INV_OCC, a.BI_OCC, b.bi as BI_OCC_idx
    from (
        select ROW_NUMBER() over(order by BI_OCC,INV_OCC) as occ_id, INV_OCC, BI_OCC
        from (
            select distinct INV_OCC, BI_OCC from huvoccus
        ) a
    )a
    join (
        select ROW_NUMBER() over(order by BI_OCC) as bi, BI_OCC
        from (select distinct BI_OCC from huvoccus) a
    ) b on a.BI_OCC= b.BI_OCC""").to_csv('working/help_occ_lu.csv', index=False, header=True)

### 3. help_cc_lu

In [12]:
df =pd.DataFrame()
for cc in ["MAPCCLSSIF", "MAPCCTIER1", "MAPCCTIER2", "MAPCCTIER3"]:
    df1  = rms_db.select(f"""
        select '{cc}' as cc_field, {cc} as cc_value, ROW_NUMBER() over(order by {cc}) as cc_code
        from (select distinct {cc} from huvccus where {cc} is not null) a""")
    df = pd.concat([df, df1], axis=0)

df.to_csv('working/help_cc_lu.csv', index=False, header=True)

### 4. pos_sm_option

In [17]:
rms_db.select("""
    select b.fid * 256 + option_num as sm_opt_id, 
        case when a.modif_num = 4 then 'RMS_Roof_Covering'
            when a.modif_num = 6 then 'RMS_Roof_Age_and_Condition'
            when a.modif_num = 7 then 'RMS_Roof_Geometry'
            when a.modif_num = 9 then 'RMS_Roof_Anchors'
            when a.modif_num = 13 then 'RMS_Cladding_Type'
            when a.modif_num = 14 then 'RMS_Roof_Sheathing_Attachment'
            when a.modif_num = 19 then 'RMS_Opening_Protection'
            else NULL
        end feature_name,
        a.modif_num, b.modif_desc, option_num, opt_desc
    from huodscus a
    join (
        select ROW_NUMBER() over (order by modif_num) as fid, modif_num, modif_desc
        from (
              select distinct modif_num, modif_desc from humdscus) a
            ) b on a.modif_num =b.modif_num
    order by fid, option_num""").to_csv('working/pos_sm_option.csv', index=False, header=True)

